In [1]:
import datetime
import math
import os
import time
import pandas as pd
from la_metro import LAMetro

try:
    BASE_DIR = os.path.abspath('')
except NameError:
    print("Warning, directories can change downstream.")
    BASE_DIR = os.getcwd()

print(BASE_DIR)


/Users/irahorecka/Desktop/Harddrive_Desktop/Python/LA_metro_map


In [2]:
"""
A function to convert standard latitude
and longitude coordinates to a mercator
coordinate system
"""
def mercator(lat, lon):
    r_major = 6378137.000
    merc_lon = r_major * math.radians(lon)
    try:
        scale = merc_lon/lon
    except ZeroDivisionError:
        return None
    merc_lat = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + lat * (math.pi/180.0)/2.0)) * scale

    return (merc_lon, merc_lat)


In [3]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models.annotations import Title
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.io import export_png

"""
A function to plot mercator coordinates over
Los Angeles map. Alter point alpha and color
to your desire.
"""
def plot_bokeh(x_list, y_list, count):
    # Set map boundaries (mercator) and axis type
    p = figure(x_range=(-13220703.47, -13124567.96),
               y_range=(3985860.84, 4082774.46),
               x_axis_type="mercator", y_axis_type="mercator")
    
    # Tile Source for CartoDB Tile Service
    p.add_tile(CARTODBPOSITRON)
    # Set styling for Metro bus coordinate
    p.circle(x=y_list,
             y=x_list,
             color='purple',
             fill_alpha=0.1)
    
    # Set chart title - date + time
    title = Title()
    title.text = f"LA Metro Buses - {datetime.datetime.now().strftime('%m/%d/%Y  %H:%M')}"
    p.title = title
    
    # For displaying map in notebook - comment out to avoid cluttering
    # output_notebook()
    # show(p)
    
    # Export as .png
    export_png(p, filename=f"plot{count}.png")
    

In [4]:
# Make client instance
client = LAMetro()

"""
Continually export bus coordinate images
every 10 seconds to ./map_figs
"""
def main():
    os.chdir("{}/map_figs".format(BASE_DIR))
    fig_count = 0
    try:
        while True:
            t0 = time.time()

            # Fetch all LA Metro bus information
            try:
                all_bus = client.bus.vehicles.all()
            except TypeError:
                continue

            lat_list = []
            long_list = []
            # Gather lat, long of every bus and convert
            # to mercator coord
            for i in all_bus:
                tot_val = mercator(i.y, i.x)
                if not tot_val:
                    continue
                lat_list.append(tot_val[1])
                long_list.append(tot_val[0])

            # Plot mercator coord on map
            plot_bokeh(lat_list, long_list, fig_count)

            fig_count += 1
            t1 = time.time()
            # Induce 10 second wait time
            if 10 - (t1 - t0) <= 0:
                continue
            time.sleep(10 - (t1 - t0))
    
    # Handle abort or connection error
    except KeyboardInterrupt:
        print("Exiting protocol...")
    except Exception as error:
        print("{e}.\n\nPlease restart kernel.".format(e=error))

    finally:
        os.chdir(BASE_DIR)


In [5]:
main()

('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')).

Please restart kernel.
